In [17]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def tumor_coverage_1(nrrd_file, center, radii):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii[0] / Spacing[0], radii[1] / Spacing[1], radii[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()

    # Align the iceball and tumor images
    sp = sphere1.GetOutput()
    sp.SetOrigin(image.GetOrigin())

    # Logic AND operation between iceball and tumor
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(image)
    logic.SetInput2Data(sp)
    logic.SetOperationToAnd()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate sum
    sc = logic.GetOutput().GetPointData().GetScalars()
    a = vtk_to_numpy(sc)
    return numpy.sum(a)

In [39]:
import vtk
import numpy as np
from vtk.util.numpy_support import vtk_to_numpy

def perturbed_tumor_percentage_GPT(nrrd_file, simulate, std_x, std_y, std_z, center, radii):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()
    
    image = reader.GetOutput()

    # Create the ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii[0] / Spacing[0], radii[1] / Spacing[1], radii[2] / Spacing[2])
    
    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()

    # Align the iceball and tumor images
    sp = sphere1.GetOutput()
    sp.SetOrigin(image.GetOrigin())

    counter = 0  # Initialize counter outside the loop

    for i in range(simulate):
        # Perturb Center
        x = center[0]
        y = center[1]
        z = center[2]

        # Generate random perturbations from Gaussian distributions
        perturb_x = np.random.normal(loc=0, scale=std_x)
        perturb_y = np.random.normal(loc=0, scale=std_y)
        perturb_z = np.random.normal(loc=0, scale=std_z)

        # Apply perturbations to the coordinates
        x_perturbed = x + perturb_x
        y_perturbed = y + perturb_y
        z_perturbed = z + perturb_z
        
        center = [x_perturbed, y_perturbed, z_perturbed]

        # Logic AND operation between iceball and tumor
        logic = vtk.vtkImageLogic()
        logic.SetInput1Data(image)
        logic.SetInput2Data(sp)
        logic.SetOperationToAnd()
        logic.SetOutputTrueValue(1)
        logic.Update()

        # Extract scalar data and calculate iceball coverage
        image_data = vtk_to_numpy(image.GetPointData().GetScalars())
        logic_data = vtk_to_numpy(logic.GetOutput().GetPointData().GetScalars())
    
        image_coverage = np.sum(image_data)  # Total coverage of the original image
        iceball_coverage = np.sum(logic_data)  # Coverage of the iceball within the image
        
        if iceball_coverage / image_coverage < 1.0:
            counter = counter + 1

    return counter / simulate  # Return the percentage of tumor coverage


In [19]:
nrrd_file = "path_to_your_nrrd_file.nrrd"
simulate = 1000
std_x = 1.0
std_y = 1.0
std_z = 1.0
center = (100, 100, 100)
radii = (10, 15, 8)

result = perturbed_tumor_percentage_GPT(nrrd_file, simulate, std_x, std_y, std_z, center, radii)
print(f"Tumor Coverage Percentage: {result * 100:.2f}%")


2023-07-18 15:00:23.478 (18477.588s) [         2221C71]      vtkNrrdReader.cxx:350    ERR| vtkNrrdReader (0x7f9b94704c40): path_to_your_nrrd_file.nrrd is not a nrrd file.
2023-07-18 15:00:23.478 (18477.588s) [         2221C71]       vtkExecutive.cxx:741    ERR| vtkCompositeDataPipeline (0x600003cc0000): Algorithm vtkNrrdReader (0x7f9b94704c40) returned failure for request: vtkInformation (0x6000024c85a0)
  Debug: Off
  Modified Time: 6298
  Reference Count: 1
  Registered Events: (none)
  Request: REQUEST_INFORMATION
  ALGORITHM_AFTER_FORWARD: 1
  FORWARD_DIRECTION: 0


2023-07-18 15:00:23.478 (18477.589s) [         2221C71]       vtkImageData.cxx:1547   ERR| vtkImageData (0x7f9b94706c40): GetScalarPointer: Pixel (0, 0, 0) not in memory.
 Current extent= (0, -1, 0, -1, 0, -1)


AttributeError: 'NoneType' object has no attribute 'GetDataType'

In [20]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center = [123, 120, 22]
radii = [100.0, 100.0, 100.0]
sum_result = tumor_coverage_1(nrrd_file, center, radii)
print(sum_result)

1780


In [21]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def tumor_coverage_2(nrrd_file, center1, radii1, center2, radii2):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the 1st ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center1)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii1[0] / Spacing[0], radii1[1] / Spacing[1], radii1[2] / Spacing[2])
    
    # Create the 2nd ellipsoid source
    sphere2 = vtk.vtkImageEllipsoidSource()
    sphere2.SetOutputScalarTypeToShort()
    sphere2.SetCenter(center2)
    Spacing = image.GetSpacing()
    sphere2.SetRadius(radii2[0] / Spacing[0], radii2[1] / Spacing[1], radii2[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()
    sphere2.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere2.Update()

    # Align the 1st iceball and tumor images
    sp1 = sphere1.GetOutput()
    sp1.SetOrigin(image.GetOrigin())
    
    # Align the 2nd iceball and tumor images
    sp2 = sphere2.GetOutput()
    sp2.SetOrigin(image.GetOrigin())

    # Logic AND operation between 1st iceball and tumor
    logic1 = vtk.vtkImageLogic()
    logic1.SetInput1Data(image)
    logic1.SetInput2Data(sp1)
    logic1.SetOperationToAnd()
    logic1.SetOutputTrueValue(1)
    logic1.Update()
    
    # Logic AND operation between 2nd iceball and tumor
    logic2 = vtk.vtkImageLogic()
    logic2.SetInput1Data(image)
    logic2.SetInput2Data(sp2)
    logic2.SetOperationToAnd()
    logic2.SetOutputTrueValue(1)
    logic2.Update()

    # Logic OR between (1st iceball and tumor) AND (2nd iceball and tumor)
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(logic1.GetOutput())
    logic.SetInput2Data(logic2.GetOutput())
    logic.SetOperationToOr()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate sum
    sc = logic.GetOutput().GetPointData().GetScalars()
    a = vtk_to_numpy(sc)
    return numpy.sum(a)

1780


In [15]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def tumor_coverage_3(nrrd_file, center1, radii1, center2, radii2, center3, radii3):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the 1st ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center1)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii1[0] / Spacing[0], radii1[1] / Spacing[1], radii1[2] / Spacing[2])
    
    # Create the 2nd ellipsoid source
    sphere2 = vtk.vtkImageEllipsoidSource()
    sphere2.SetOutputScalarTypeToShort()
    sphere2.SetCenter(center2)
    sphere2.SetRadius(radii2[0] / Spacing[0], radii2[1] / Spacing[1], radii2[2] / Spacing[2])

    # Create the 3rd ellipsoid source
    sphere3 = vtk.vtkImageEllipsoidSource()
    sphere3.SetOutputScalarTypeToShort()
    sphere3.SetCenter(center3)
    sphere3.SetRadius(radii3[0] / Spacing[0], radii3[1] / Spacing[1], radii3[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()
    sphere2.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere2.Update()
    sphere3.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere3.Update()

    # Align the 1st iceball and tumor images
    sp1 = sphere1.GetOutput()
    sp1.SetOrigin(image.GetOrigin())
    
    # Align the 2nd iceball and tumor images
    sp2 = sphere2.GetOutput()
    sp2.SetOrigin(image.GetOrigin())

    # Align the 3rd iceball and tumor images
    sp3 = sphere3.GetOutput()
    sp3.SetOrigin(image.GetOrigin())

    # Logic AND operation between 1st iceball and tumor
    logic1 = vtk.vtkImageLogic()
    logic1.SetInput1Data(image)
    logic1.SetInput2Data(sp1)
    logic1.SetOperationToAnd()
    logic1.SetOutputTrueValue(1)
    logic1.Update()
    
    # Logic AND operation between 2nd iceball and tumor
    logic2 = vtk.vtkImageLogic()
    logic2.SetInput1Data(image)
    logic2.SetInput2Data(sp2)
    logic2.SetOperationToAnd()
    logic2.SetOutputTrueValue(1)
    logic2.Update()

    # Logic AND operation between 3rd iceball and tumor
    logic3 = vtk.vtkImageLogic()
    logic3.SetInput1Data(image)
    logic3.SetInput2Data(sp3)
    logic3.SetOperationToAnd()
    logic3.SetOutputTrueValue(1)
    logic3.Update()

    # Logic AND between ((1st iceball and tumor) AND (2nd iceball and tumor)) AND (3rd iceball and tumor)
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(logic1.GetOutput())
    logic.SetInput2Data(logic2.GetOutput())
    logic.SetOperationToAnd()
    logic.Update()
    logic.SetInput1Data(logic.GetOutput())  # Use the result from previous AND operation
    logic.SetInput2Data(logic3.GetOutput())
    logic.SetOperationToAnd()
    logic.Update()

    # Extract scalar data and calculate sum
    sc = logic.GetOutput().GetPointData().GetScalars()
    a = vtk_to_numpy(sc)
    return numpy.sum(a)

In [16]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center1 = [123, 120, 22]
radii1 = [10.0, 10.0, 10.0]
center2 = [123, 120, 22]
radii2 = [10.0, 10.0, 10.0]
center3 = [123, 120, 22]
radii3 = [10.0, 10.0, 10.0]
sum_result = tumor_coverage_3(nrrd_file, center1, radii1, center2, radii2, center3, radii3)
print(sum_result)

405960


In [37]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def iceball_coverage_1(nrrd_file, center, radii):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii[0] / Spacing[0], radii[1] / Spacing[1], radii[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()

    # Align the iceball and tumor images
    sp = sphere1.GetOutput()
    sp.SetOrigin(image.GetOrigin())

    # Logic AND operation between iceball and tumor
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(image)
    logic.SetInput2Data(sp)
    logic.SetOperationToAnd()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate iceball coverage
    image_data = vtk_to_numpy(image.GetPointData().GetScalars())
    logic_data = vtk_to_numpy(logic.GetOutput().GetPointData().GetScalars())
    
    image_coverage = numpy.sum(image_data)  # Total coverage of the original image
    iceball_coverage = numpy.sum(logic_data)  # Coverage of the iceball within the image
    
    return iceball_coverage / image_coverage

In [38]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center = [121, 118, 22]
radii = [18.0, 10.0, 18.0]
coverage_ratio = iceball_coverage_1(nrrd_file, center, radii)
print("Iceball coverage ratio:", coverage_ratio)

Iceball coverage ratio: 1.0


In [35]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def iceball_coverage_2(nrrd_file, center1, radii1, center2, radii2):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()
    
    # Create the 1st ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center1)  # Fixed variable name from center to center1
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii1[0] / Spacing[0], radii1[1] / Spacing[1], radii1[2] / Spacing[2])
    
    # Create the 2nd ellipsoid source
    sphere2 = vtk.vtkImageEllipsoidSource()
    sphere2.SetOutputScalarTypeToShort()
    sphere2.SetCenter(center2)  # Fixed variable name from center to center2
    Spacing = image.GetSpacing()
    sphere2.SetRadius(radii2[0] / Spacing[0], radii2[1] / Spacing[1], radii2[2] / Spacing[2])
    
    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()
    
    # Align the iceball and tumor images
    sp1 = sphere1.GetOutput()
    sp1.SetOrigin(image.GetOrigin())
    sp2 = sphere2.GetOutput()
    sp2.SetOrigin(image.GetOrigin())

    # Logic AND operation between 1st iceball and tumor
    logic1 = vtk.vtkImageLogic()
    logic1.SetInput1Data(image)
    logic1.SetInput2Data(sp1)
    logic1.SetOperationToAnd()
    logic1.SetOutputTrueValue(1)
    logic1.SetOutputScalarTypeToShort()  # Set the scalar type to "short"

    logic1.Update()
    
    # Logic AND operation between 2nd iceball and tumor
    logic2 = vtk.vtkImageLogic()
    logic2.SetInput1Data(image)
    logic2.SetInput2Data(sp2)  
    logic2.SetOperationToAnd()
    logic2.SetOutputTrueValue(1)
    logic2.SetOutputScalarTypeToShort()
    logic2.Update()
    
    # Logic OR operation between (1st iceball and tumor) AND (2nd iceball and tumor)
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(logic1.GetOutput())
    logic.SetInput2Data(logic2.GetOutput())
    logic.SetOperationToOr()
    logic.SetOutputTrueValue(1)
    logic.SetOutputScalarTypeToShort()
    logic.Update()
    
    # Extract scalar data and calculate iceball coverage
    image_data = vtk_to_numpy(image.GetPointData().GetScalars())
    logic_data = vtk_to_numpy(logic.GetOutput().GetPointData().GetScalars())
    
    image_coverage = numpy.sum(image_data)  # Total coverage of the original image
    iceball_coverage = numpy.sum(logic_data)  # Coverage of the iceball within the image
    
    return iceball_coverage / image_coverage

In [36]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center1 = [112, 121, 22]
radii1 = [10.0, 10.0, 10.0]
center2 = [126, 122, 22]
radii2 = [10.0, 10.0, 10.0]
coverage_ratio = iceball_coverage_2(nrrd_file, center1, radii1, center2, radii2)
print("Iceball coverage ratio:", coverage_ratio)

AttributeError: 'vtkmodules.vtkImagingMath.vtkImageLogic' object has no attribute 'SetOutputScalarTypeToShort'

In [34]:
import os

myfile = "/Users/alvarocuervo/Desktop/teste-label.nrrd"  # Mac absolute path
file_exists = os.path.isfile(myfile)

if file_exists:
    print("The file exists.")
else:
    print("The file does not exist or cannot be accessed.")


The file exists.


In [ ]:
# Python program to explain os.getcwd() method
# importing os module
import os
	
# Get the current working
# directory (CWD)
cwd = os.getcwd()
	
# Print the current working
# directory (CWD)
print("Current working directory:", cwd)
